In [235]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Brainstorm:

1. Before Training: Use PCA to reduce the number of features (Low priority)

    1.1 Things such as the dog ID should not make a difference

2. Training: 

    2.1 Use deep neutral network to train a model and make predictions
    
    2.2 Use boosting and bagging to improve the model (Low priority)

3. Things to consider:

    3.1 Need to choose loss functions
    
    3.2 We should numericalize and normalize all features. Need to come up with ways to represent things such as "date of birth" and "N/A" data entries as numbers.
    


# STEP 1: Data Loading and Cleaning

In [236]:
# Choose from LF, LH, RF, RH 
leg = "RH"

x_file_name = leg + "_train.csv"

x_lf = pd.read_csv(x_file_name)

y_lf = x_lf[leg]



In [237]:
def clean_up_my_data_bro(x_lf, leg = "POTATO", data_type = "test"):
    
    if data_type=="test":
        x_lf = x_lf.drop(labels=["id","dob","forceplate_date"], axis=1) # these should not matter
    elif data_type=="train":
        x_lf = x_lf.drop(labels=["id","dob","forceplate_date",leg], axis=1) # these should not matter
    else:
        print("wrong data_type bro")
        
    
    # if gait is walk then 1 else -1
    for i in range(0,len(x_lf['gait'])):
        if x_lf['gait'][i] == "Walk":
            x_lf['gait'][i] = 1
        else:
            x_lf['gait'][i]= -1
    
    
    for i in range(0,len(x_lf['speed'])):
        if x_lf['speed'][i] == "Not able to walk":
            x_lf['speed'][i] = 0
        if x_lf['speed'][i] == "Not able to trot":
            x_lf['speed'][i] = -1

    x_lf = np.array(x_lf)
            
    col_remove = []
    print("x_training col number ", len(x_lf[0]))
    
    
    for j in range(0,len(x_lf[0])):

        col_sum = 0
        col_count =0

        for i in range(0,len(x_lf)):
            
            # More Cleaning Lmao
            if j==185:
                if x_lf[i][j]=="Trot":
                    x_lf[i][j]=1
            
            #if j==185: # NOT ONLY FOR 185
            if x_lf[i][j]=="Not able to trot":
                x_lf[i][j]=-1
        
            if x_lf[i][j]=="no valid trials":
                x_lf[i][j]= float("nan") # CHECK THIS
        
            if x_lf[i][j] == "No Data" or x_lf[i][j]=="no data":
                x_lf[i][j] = float("nan")


            if isinstance(x_lf[i][j], str)==True:
                #print("string found at " + str(i) + " " +str(j))
                x_lf[i][j] = float(x_lf[i][j])    


            
            if np.isnan(x_lf[i][j])==False:
                col_sum += x_lf[i][j]
                col_count += 1

            
            
    #         Idea 1: only count not nan values
    #         else:
    #             col_sum += x_lf[i][j]
    #             col_count += 1
            # Idea 2: count them but if variance is zero, remove column

        col_mean = col_sum/col_count
        col_var = 0
        
        # this loop set nan to the mean (so 0 after normalized) and calculates variance
        
        for i in range(0,len(x_lf)):
            if np.isnan(x_lf[i][j]):
                x_lf[i][j]=col_mean
            col_var += (x_lf[i][j]-col_mean)**2
            

        col_std = (col_var/float(col_count))**(1/2)

        if col_std == 0:
            col_remove.append(j) 
            col_std = 1
            print("removed column ", j, " bc all data are the same")

        x_lf[:,j] -= col_mean
        x_lf[:,j] /= col_std
    
    
    
    # I MADE THE APPROXIMATION THAT NAN=-1, CAN IMPROVE LATER BY LOOKING AT SPECIFIC COLUMNS
    

    for i in col_remove:
        x_lf = np.delete(x_lf,obj=i,axis=1)

    print("x_lf and y_lf now has " + str(len(x_lf[0])) + " columns")


    x_lf2 = x_lf # Cleaned_Data


    return x_lf2


In [238]:
# x_lf = x_lf.drop(labels=["id","dob","forceplate_date","RH"], axis=1) # these should not matter


In [239]:
# for i in x_lf.columns:
#     print(i)



In [240]:
x_lf2 = clean_up_my_data_bro(x_lf, leg=leg, data_type = "train")

x_training col number  367
removed column  3  bc all data are the same
removed column  185  bc all data are the same
x_lf and y_lf now has 365 columns


<ipython-input-237-9aae06521892>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_lf['gait'][i] = 1


In [241]:
# for i in x_lf2:
#     for j in i:
#         print(j)

In [242]:
# let labels be -1 and 1
for i in range(0,len(y_lf)):
    if y_lf[i]==0:
        y_lf[i]=-1

y_lf2 = np.array(y_lf)

<ipython-input-242-7e50fc914804>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_lf[i]=-1


# STEP 2: Train a Model

In [243]:
#<GRADED>
class TreeNode(object):
    """Tree class.
    
    (You don't need to add any methods or fields here but feel
    free to if you like. Our tests will only reference the fields
    defined in the constructor below, so be sure to set these
    correctly.)
    """
    
    def __init__(self, left, right, parent, cutoff_id, cutoff_val, prediction):
        self.left = left
        self.right = right
        self.parent = parent
        self.cutoff_id = cutoff_id
        self.cutoff_val = cutoff_val
        self.prediction = prediction
#</GRADED>

In [244]:
#<GRADED>
def sqsplit(xTr,yTr,weights=[]):
    """Finds the best feature, cut value, and loss value.
    
    Input:
        xTr:     n x d matrix of data points
        yTr:     n-dimensional vector of labels
        weights: n-dimensional weight vector for data points
    
    Output:
        feature:  index of the best cut's feature
        cut:      cut-value of the best cut
        bestloss: loss of the best cut
    """
    
    def compute_loss(w, mu, s):
        return s - (mu ** 2) / w 
    
    N,D = xTr.shape
    assert D > 0 # must have at least one dimension
    assert N > 1 # must have at least two samples
    if weights == []: # if no weights are passed on, assign uniform weights
        weights = np.ones(N)
    weights = weights/sum(weights) # Weights need to sum to one (we just normalize them)
    bestloss = np.inf
    feature = np.inf
    cut = np.inf
    
    for d in range(D):
        ii = xTr[:,d].argsort() # sort data along the dth dimension
        xs = xTr[ii,d] # sorted feature values
        ws = weights[ii] # sorted weights
        ys = yTr[ii] # sorted labels
        
        # Initialize constants
        sL=0.0 # mean squared label on left side
        muL=0.0 # mean label on left side
        wL=0.0 # total weight on left side
        sR=ws.dot(ys**2) #mean squared label on right 
        muR=ws.dot(ys) # mean label on right
        wR=sum(ws) # weight on right
         
        idif = np.where(np.abs(np.diff(xs, axis=0)) > np.finfo(float).eps * 100)[0]
        pj = 0

        for j in idif:
            
            ys_slice = ys[pj: j + 1]
            ws_slice = ws[pj: j + 1]
            ws_to_move = np.sum(ws_slice)
            mus_to_move = np.dot(ys_slice, ws_slice)
            ss_to_move = np.dot(ws_slice, np.power(ys_slice, 2))
            wL += ws_to_move
            wR -= ws_to_move
            muL += mus_to_move
            muR -= mus_to_move
            sL += ss_to_move
            sR -= ss_to_move
            pj = j + 1 # don't forget to change pj in this case
            lossL, lossR = compute_loss(wL, muL, sL), compute_loss(wR, muR, sR)
            loss = lossL + lossR
            if loss < bestloss:
                feature = d
                cut = (xs[j] + xs[j + 1]) / 2 # adding eps to ensure the last point is in the left partition
                bestloss = loss
        
    assert feature != np.inf and cut != np.inf
    
    return feature, cut, bestloss
    

#</GRADED>

#<GRADED>
def cart(xTr,yTr,depth=np.inf,weights=None):
    """Builds a CART tree.
    
    The maximum tree depth is defined by "maxdepth" (maxdepth=2 means one split).
    Each example can be weighted with "weights".

    Args:
        xTr:      n x d matrix of data
        yTr:      n-dimensional vector
        maxdepth: maximum tree depth
        weights:  n-dimensional weight vector for data points

    Returns:
        tree: root of decision tree
    """
    n,d = xTr.shape
    if weights is None:
        w = np.ones(n) / float(n)
    else:
        w = weights
    
    def build_cart(xTr, yTr, depth, weights, parent):
        n, d = xTr.shape
        pred = np.mean(yTr)
 
        
        label_same = np.all(yTr==yTr[0])
        feature_same = np.all(xTr==xTr[0])
        if depth <= 1 or label_same or feature_same:
            return TreeNode(None, None, parent, None, None, pred)
        else:
            feature, cut, _ = sqsplit(xTr,yTr,weights)
            ii = xTr[:,feature].argsort()
            xs = xTr[ii,:] 
            ws = weights[ii]
            ys = yTr[ii]
            lefts = np.where(xs[:,feature] <= cut)[0]
            rights = np.where(xs[:, feature] > cut)[0]
            xTrL, wsL, yTrL = xs[lefts,:], ws[lefts], ys[lefts]
            xTrR, wsR, yTrR = xs[rights,:], ws[rights], ys[rights]
            depth -= 1
            node = TreeNode(None, None, parent, feature, cut, pred)
            left_child = build_cart(xTrL, yTrL, depth, wsL, node)
            right_child = build_cart(xTrR, yTrR, depth, wsR, node)
            node.left = left_child
            node.right = right_child
            return node
            
    return build_cart(xTr, yTr, depth, w, None)
    
    
#</GRADED>

#<GRADED>
def forest(xTr, yTr, m, maxdepth=np.inf):
    """Creates a random forest.
    
    Input:
        xTr:      n x d matrix of data points
        yTr:      n-dimensional vector of labels
        m:        number of trees in the forest
        maxdepth: maximum depth of tree
        
    Output:
        trees: list of TreeNode decision trees of length m
    """
    
    n, d = xTr.shape
    trees = []
    
    for i in range(0,m):
        index = np.random.randint(low=0, high=n , size=n) # goes from 0 to n-1, n samples
        
        sampled_xTr = xTr[index]
        sampled_yTr = yTr[index]
        
        result = cart(sampled_xTr, sampled_yTr, depth = maxdepth)
        
        trees.append(result)
        
    return trees
#</GRADED>

#<GRADED>
def evaltree(root,xTe):
    """Evaluates xTe using decision tree root.
    
    Input:
        root: TreeNode decision tree
        xTe:  n x d matrix of data points
    
    Output:
        pred: n-dimensional vector of predictions
    """
    
    def evaltree_point(root, x):
        if root.left is None:
            return root.prediction
        else:
            if x[root.cutoff_id] < root.cutoff_val:
                return evaltree_point(root.left, x)
            else:
                return evaltree_point(root.right, x)
    
    assert root is not None
    n = xTe.shape[0]
    pred = np.zeros(n)
    
    for i in range(n):
        pred[i] = evaltree_point(root, xTe[i,:])

    return pred
#</GRADED>

#<GRADED>
def evalforest(trees, X, alphas=None):
    """Evaluates X using trees.
    
    Input:
        trees:  list of TreeNode decision trees of length m
        X:      n x d matrix of data points
        alphas: m-dimensional weight vector
        
    Output:
        pred: n-dimensional vector of predictions
    """
    m = len(trees)
    n,d = X.shape
    if alphas is None:
        alphas = np.ones(m) / m
            
    pred = np.zeros(n)
    
    for i in range(m):
        preds = evaltree(trees[i], X)
        pred += alphas[i] * preds
    
    return pred
#</GRADED>

In [245]:
test_file_name = leg + "_test.csv"

t_lf = pd.read_csv(test_file_name)

t_id_arr = np.array(t_lf['id'])

t_lf2 = clean_up_my_data_bro(t_lf, leg, data_type="test")

<ipython-input-237-9aae06521892>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_lf['gait'][i] = 1
<ipython-input-237-9aae06521892>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_lf['speed'][i] = 0


x_training col number  367
removed column  3  bc all data are the same
removed column  185  bc all data are the same
x_lf and y_lf now has 365 columns


In [246]:
maxdepth = 5
m=75

my_forest = forest(x_lf2, y_lf2, m, maxdepth=maxdepth)

<ipython-input-244-ff6721efe2f5>:22: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if weights == []: # if no weights are passed on, assign uniform weights


In [247]:
pred_lf = evalforest(my_forest, t_lf2, None)

In [248]:
final_lf  = np.sign(pred_lf)

for i in range(0,len(final_lf)):
    final_lf[i]=int(float(final_lf[i]))
    if final_lf[i]==-1:
        final_lf[i]=0

In [249]:
final_lf = final_lf.astype(int)

# STEP 3: Make Output Files

In [250]:
import csv

rows = zip(t_id_arr, final_lf)


In [251]:
# Write the tuples to a CSV file

csv_file_name = leg + "_test_labels.csv"

with open(csv_file_name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id', leg]) # write header row
    writer.writerows(rows)

In [254]:
def make_sub():

    """
    This is a file that will help you convert your individual predictions to the final prediction. 
    In the same directory as this file, you should have the following 4 files:
      - LF_test_labels.csv - with at least two columns, 'id' and 'LF'
      - LH_test_labels.csv - with at least two columns, 'id' and 'LH'
      - RF_test_labels.csv - with at least two columns, 'id' and 'RF'
      - RH_test_labels.csv - with at least two columns, 'id' and 'RH'

    Running this script will convert these four files into a single CSV file, submission.csv, by
    mutating the IDs so that they also include the leg that is being checked.
    """

    legs = ["LF", "LH", "RF", "RH"]

    dfs = []

    for leg in legs:
        # read in the file
        test_prediction = pd.read_csv(f"{leg}_test_labels.csv")
        # append the abbreviation for the leg
        test_prediction['id'] = test_prediction['id'].astype(str) + f"_{leg}"
        # rename the label column
        test_prediction['label'] = test_prediction[leg]
        # exclude any potential additional columns
        dfs.append(test_prediction[['id', 'label']])

    final_df = pd.concat(dfs)
    final_df.to_csv("submission.csv", index=False)



In [255]:
make_sub()